In [1]:
import pandas as pd 
import glob
import os
import numpy as np

In [2]:
PATH_DATA = "../data"

## Loading Phase

In [3]:
filenames = glob.glob(os.path.join(PATH_DATA, "*.csv"))
print(filenames)
df = pd.concat((pd.read_csv(f) for f in filenames))
print(df.shape)
df.head()

['../data/IRAhandle_tweets_4.csv', '../data/IRAhandle_tweets_5.csv', '../data/IRAhandle_tweets_7.csv', '../data/IRAhandle_tweets_6.csv', '../data/IRAhandle_tweets_2.csv', '../data/IRAhandle_tweets_3.csv', '../data/IRAhandle_tweets_1.csv', '../data/IRAhandle_tweets_8.csv', '../data/IRAhandle_tweets_9.csv']
(2973371, 15)


,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1.674084e+09,GAB1ALDANA,People are too toxic. I think I have people po...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2150,RETWEET,Hashtager,0,1,HashtagGamer
1,1.674084e+09,GAB1ALDANA,#NowPlaying Don't Shoot (I'm a Man) by @DEVO -...,United States,English,7/30/2016 20:15,7/30/2016 20:15,3395,2014,2146,RETWEET,Hashtager,0,1,HashtagGamer
2,1.674084e+09,GAB1ALDANA,the 'I'm the most boring person in the world' ...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2159,RETWEET,Hashtager,0,1,HashtagGamer
3,1.674084e+09,GAB1ALDANA,#MyAchillesHeel slippery floors https://t.co/R...,United States,Norwegian,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2160,RETWEET,Hashtager,0,1,HashtagGamer
4,1.674084e+09,GAB1ALDANA,#MyAchillesHeel Boring narcissists.....nothing...,United States,English,7/30/2016 20:16,7/30/2016 20:16,3395,2013,2158,RETWEET,Hashtager,0,1,HashtagGamer


In [6]:
df['retweet'].unique()

array([1, 0])

In [16]:
float(df.loc[df['region'] == 'United States','region'].count())/float(df['region'].count())

0.6934938715370541

In [4]:
df.dtypes

external_author_id    float64
author                 object
content                object
region                 object
language               object
publish_date           object
harvested_date         object
following               int64
followers               int64
updates                 int64
post_type              object
account_type           object
new_june_2018           int64
retweet                 int64
account_category       object
dtype: object

In [5]:
df.describe(include="all")

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
count,2.973367e+06,2973371,2973370,2964528,2973371,2973371,2973371,2.973371e+06,2.973371e+06,2.973371e+06,1310946,2973008,2.973371e+06,2.973371e+06,2973371
unique,NaN,2848,2365942,36,56,896684,906316,NaN,NaN,NaN,2,20,NaN,NaN,8
top,NaN,EXQUOTE,В городе Сочи. Олимпиада – праздник или стихий...,United States,English,8/16/2017 1:29,3/22/2016 17:35,NaN,NaN,NaN,RETWEET,Russian,NaN,NaN,NonEnglish
freq,NaN,59652,670,2055882,2128963,202,1333,NaN,NaN,NaN,1270702,721191,NaN,NaN,837725
mean,1.296128e+17,NaN,NaN,NaN,NaN,NaN,NaN,3.433524e+03,7.018913e+03,1.049756e+04,NaN,NaN,2.078735e-01,4.408955e-01,NaN
std,3.036341e+17,NaN,NaN,NaN,NaN,NaN,NaN,5.609881e+03,1.458463e+04,1.768729e+04,NaN,NaN,4.057859e-01,4.964945e-01,NaN
min,3.497640e+07,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000e+00,-1.000000e+00,-1.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,NaN
25%,1.930748e+09,NaN,NaN,NaN,NaN,NaN,NaN,3.270000e+02,3.200000e+02,1.787000e+03,NaN,NaN,0.000000e+00,0.000000e+00,NaN
50%,2.581835e+09,NaN,NaN,NaN,NaN,NaN,NaN,1.499000e+03,1.274000e+03,4.333000e+03,NaN,NaN,0.000000e+00,0.000000e+00,NaN
75%,3.254274e+09,NaN,NaN,NaN,NaN,NaN,NaN,4.730000e+03,1.060000e+04,1.234100e+04,NaN,NaN,0.000000e+00,1.000000e+00,NaN


In [23]:
df['post_type'].unique()

array(['RETWEET', nan, 'QUOTE_TWEET'], dtype=object)

In [24]:
(df['post_type']=='RETWEET').sum()

1270702

In [25]:
(df['post_type']=='QUOTE_TWEET').sum()

40244

In [35]:
df['post_type'].isnull().sum()

1662425